# Notebook to analyze period gradient & phase gradient in spreadouts
Paul Gerald Layague Sanchez (paul.sanchez@embl.de or pglsanchez@gmail.com) and Takehito Tomita (takehito.tomita@embl.de)

Last updated: 20200528 1856H

This script analyzes the period gradient and phase gradient in spreadouts. The input files for this script are:

* blurred intensity movie (dorsal side is up)  
* period wavelet movie (dorsal side is up)
* phase wavelet movie (dorsal side is up)

### Import modules here:

In [ ]:
import sys,os
from sys import argv, stdout
from os.path import expanduser
from pathlib import Path

import numpy as np
from os import path,walk

sys.path.append(os.path.expanduser("~/PSM/progs/lib/"))

from skimage import io

from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors

import pandas as pd

import seaborn as sns

sns.set_style("ticks")
sns.despine()
sns.set_context("talk", font_scale = 1.5)

chosen_colormap = plt.get_cmap('Blues') # for period gradient at different positions along 
chosen_colormap2 = 'viridis' # for evolution of period gradient for multiple samples

from sklearn import  linear_model
from sklearn.metrics import mean_squared_error, r2_score

from wavelet_analysis import *

### Specify parameters here:

In [ ]:
diameter = 50
dt = 10 # sampling time, in minutes
frame_interval = 5 # interval between frames to be considered
frame_interval_kymo = 1 # interval between frames for kymos
position_kymo = 256 # pixel position along vertical/DV axis for making kymo
threshold = 0.15 # for masking
min_period = 100 # minimum period for calculating true mean
start_position = 231 # start pixel position along vertical/DV axis
stop_position = 281 # stop pixel position along vertical/DV axis + 1
position_sampling = 1 # sampling interval between start_position and stop_position
T_c = 240 # cut off period for mouse spreadouts in minutes, removes low frequency (high period) trends -- for wavelet analysis
offset = 0 # offset on start of experiment, if any
characters = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z'] # for plots with sample names and numbers

### Start defining functions here:

The following function detects zero values in an [N,1] array, and extracts only the nonzero values. It returns an [M,2] array, where the first column has original indices of the nonzero values, and the second column has the nonzero values.

In [ ]:
def chop(array):
    
    ind = np.nonzero(array)
    chopped = np.array([np.array(ind)[0],array[ind]])
    
    return chopped

The following function deletes unneccessary texts from input string.

In [ ]:
# deletes unnecessary texts from input string

def strip_string(string):

    unnnecessaries = ['output_','blurred_', 'rotated_', '001-120_', 'phase_','period_','Reslice (AVRG) of ','reg_','MAX_','titration_','DE_2_','DE_3_','_P0001','_sorted-1','-1','.tif']
    for u in unnnecessaries:
        
        string = string.replace(u,'')

    return string

The next function is for generating masks from a threshold value.

In [ ]:
def img_Masking(im_blur, threshold):

    im_thresh = im_blur > threshold
    
    return im_thresh

The following function saves a series of .tif files into a multi-stack .tif file (timelapse).

In [ ]:
def figsave_to_tif(input_string, sample_num):
    wdir = os.getcwd()
    print("Working in",wdir)

    p = Path(wdir)
    figure_names = list( p.glob(input_string) )
    if len(figure_names) == 0:
        print('Found no input movie.. exiting!')
        sys.exit(1)
    figure_names.sort()

    figure_path = os.path.join( wdir, figure_names[0].name )
    figure = io.imread(figure_path, plugin="tifffile")

    mat = np.zeros([len(figure_names),figure.shape[0],figure.shape[1],figure.shape[2]], dtype = np.uint8)
    print(mat.shape)

    for index,k in enumerate(figure_names):

        figure_name = k.name # the roi movie file name

        figure_path = os.path.join( wdir, figure_name )
        print('Opening :', figure_name)
        mat[index,:,:,:] = io.imread(figure_path, plugin="tifffile")
        os.remove(figure_path)

    out_name = input_string.replace('*','')[1:]
    out_path1 = os.path.join(wdir, 'flattened_' + sample_num + out_name)
    io.imsave(out_path1, mat)
    print('written',out_path1)

This script gets trajectories from timeseries for wavelet analysis.

In [ ]:
def get_traj(col):
    traj = data_normalizedTimeseries[col]
    traj.dropna(inplace = True) # return series without null values, do operation in place (inplace=True)

    return traj

### Script for importing input files:

In [ ]:
# initialize the working directory and find input file paths

wdir = os.getcwd()
print("Working in",wdir)

p = Path(wdir)

intensity_movie_names = list( p.glob('SAMPLEDATA-blurred_rotated*tif') ) # make list of intensity movies for masking
intensity_movie_names.sort()

period_movie_names = list( p.glob('SAMPLEDATA-output_period*tif') ) # make list of period movies for period gradient
period_movie_names.sort()

phase_movie_names = list( p.glob('SAMPLEDATA-output_phase*tif') ) # make list of phase movies for phase gradient
phase_movie_names.sort()

# create a directory for output plots

if not os.path.isdir(wdir + '/kymo_pdfs'):    
    os.mkdir(wdir + '/kymo_pdfs')

In [ ]:
# generating normalized timeseries from mean global intensity

all_timeseries = []

for index,k in enumerate(intensity_movie_names):
    
    intensity_movie_name = k.name # the roi movie file name

    # open MAX movie as Mrm
    intensity_movie_path = os.path.join( wdir, intensity_movie_name )
    print('Opening :', intensity_movie_name)
    intensity_rm = io.imread(intensity_movie_path, plugin="tifffile")[:,:,:]    #tzcyx remove first channel
    
    NFrames, ydim, xdim = intensity_rm.shape

    timeseries = []

    for time in range(0,NFrames,frame_interval_kymo): # last number to specify intervals between frames to consider
        
        meanGlobalIntensity_forTimeseries = np.mean(intensity_rm[time,:,:])
        timeseries.append(meanGlobalIntensity_forTimeseries)
    
    all_timeseries.append(timeseries)

all_timeseries_df = pd.DataFrame(all_timeseries)
all_timeseries_df_transposed = all_timeseries_df.T
max_intensities = all_timeseries_df_transposed.max()
    
for i in range(len(all_timeseries_df_transposed.index)):
        
    for j in range(len(all_timeseries_df_transposed.columns)):
                            
        max_intensity = max_intensities[j]
        all_timeseries_df_transposed.iloc[i,j] = all_timeseries_df_transposed.iloc[i,j]/max_intensity

data_normalizedTimeseries = all_timeseries_df_transposed
data_normalizedTimeseries.to_csv(wdir + '/normalizedTimeseries.csv')

colors = cm.viridis(np.linspace(0, 1, len(data_normalizedTimeseries.columns)))

fig = plt.figure(figsize=(24,24))
ax = fig.add_subplot(111)
ax.set_title("NORMALIZED TIMESERIES FROM GLOBAL ROI", fontsize = 50, loc = 'center') 

for col in data_normalizedTimeseries.columns:
    
    legend = str('sample ' + str(col+1))
    ax.plot(data_normalizedTimeseries.iloc[:,col], ':', lw = 5, alpha = 0.5, color = colors[col], label = legend)
    ax.set_xlabel('TIME [min]', labelpad = 25, fontsize = 25)
    ax.set_ylabel('NORMALIZED MEAN INTENSITY [au]', labelpad = 25, fontsize = 25)
    ax.set_ylim([0,1])
    ax.set_xticklabels(ax.get_xticks()*dt)
    ax.legend(bbox_to_anchor=(1.04,0), loc="lower left")

In [ ]:
periods = linspace(100,250,200) # return 200 evenly spaced periods from 100 to 250
wAn = TFAnalyser(periods, dt, T_cut_off = T_c, vmax = 30)

allTs = pd.DataFrame(index = data_normalizedTimeseries.index) # tabular data of all periods, with index same as data index

colors = cm.viridis(np.linspace(0, 1, len(data_normalizedTimeseries.columns)))

fig = plt.figure(figsize=(24,24))
ax = fig.add_subplot(111)
ax.set_title("PERIOD VERSUS TIME", fontsize = 50, loc = 'center') 

for col in data_normalizedTimeseries.columns:
    
    traj = get_traj(col)
    wAn.compute_spectrum(traj, Plot = False)
    rdata = wAn.get_maxRidge() # has arrays for inds, power, ridge, time, and z 
    Ts = rdata['ridge'] # array for periods

    s = pd.Series(Ts) # 1d array of periods with axis labels 
    allTs[col] = s
    
    legend = str('sample ' + str(col+1))
    ax.plot(Ts, label = legend, lw = 3, alpha = 0.8, color = colors[col])
    ax.set_ylabel('PERIOD [min]', labelpad = 25, fontsize = 25)
    ax.set_ylim([100,200])  # change minimum and maximum y value, period, plotted
    ax.set_xlabel('TIME [min]', labelpad = 25, fontsize = 25)
    ax.set_xticklabels(ax.get_xticks()*dt)
    ax.yaxis.grid() # horizontal gridlines
    ax.legend(bbox_to_anchor=(1.04,0), loc="lower left")

allFreqs = 2*np.pi/allTs # frequency-time analysis based on global ROI

### ANALYZING PERIOD GRADIENT AND PHASE GRADIENT IN SPREADOUTS

The following script is for generating the intensity kymograph and a mask based on a threshold (specified in parameters). Outputs are lists containing (a) arrays of normalized intensity kymographs of all samples and (b) arrays of their corresponding masks.

In [ ]:
all_intensities_kymo_array = []
all_mask_intensitiesKymo = []

for index,k in enumerate(intensity_movie_names):
    
    intensity_movie_name = k.name # the roi movie file name

    # open MAX movie as Mrm
    intensity_movie_path = os.path.join( wdir, intensity_movie_name )
    print('Opening :', intensity_movie_name)
    intensity_rm = io.imread(intensity_movie_path, plugin="tifffile")[:,:,:]    #tzcyx remove first channel

    NFrames, ydim, xdim = intensity_rm.shape

    # make panda dataframe where rows are space and columns are time

    kymo_rows = int(xdim)
    kymo_columns = int(NFrames/frame_interval_kymo)

    intensities_forKymo = pd.DataFrame(index = range(kymo_rows), columns = range(kymo_columns))

    for time in range(0,NFrames,frame_interval_kymo): # last number to specify intervals between frames to consider
    
        for space in range(len(intensities_forKymo.index)):
            
            intensities_forKymo.iloc[space,time] = intensity_rm[time,position_kymo,space]
        
        intensities_forKymo_norm = intensities_forKymo.div(intensities_forKymo.values.max())

    # plot intensity kymograph

    sns.set(font="Helvetica")
    fig,ax = plt.subplots(figsize=(5,8))

    ax = sns.heatmap(intensities_forKymo_norm, cbar = True, cbar_kws={'label': 'normalized intensity'})
    ax.set_ylabel('SPACE [pixel]', labelpad = 15, fontsize = 15)
    ax.set_xlabel('TIME [min/10]', labelpad = 15, fontsize = 15)
    # ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 15)
    # ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 15)

    plt.show
    fig.savefig(wdir + '/kymo_pdfs/'+'intensityKymo_'+strip_string(intensity_movie_name)+'.png', bbox_inches='tight')
    
    intensities_kymo_array = np.array(intensities_forKymo_norm) # pandas dataframe to numpy array
    mask_intensitiesKymo = img_Masking(intensities_kymo_array,threshold) # mask based on intensities > threshold
    
    # plot intensity kymograph with corresponding mask

    fig, axes = plt.subplots(1, 2, figsize=(5, 15), sharex=True, sharey=True)
    ax1 = axes.ravel()

    ax1[0].imshow(intensities_kymo_array, cmap=plt.cm.magma)
    ax1[0].set_title('INTENSITY')
    ax1[0].axis('off')

    ax1[1].imshow(mask_intensitiesKymo, cmap=plt.cm.gray)
    ax1[1].set_title('MASK')
    ax1[1].axis('off')
    
    all_intensities_kymo_array.append(intensities_kymo_array)
    all_mask_intensitiesKymo.append(mask_intensitiesKymo)  

The following script is for generating the period kymograph and the masked period kymograph. Outputs are lists containing (a) arrays of period kymographs of all samples and (b) arrays of their corresponding masked period kymographs.

In [ ]:
# for single position

all_single_periods_forKymo_float_array = []
all_single_masked_periodKymo = []

for index,k in enumerate(period_movie_names):
    
    period_movie_name = k.name # the roi movie file name

    # open period movie as Mrm
    period_movie_path = os.path.join( wdir, period_movie_name )
    print('Opening :', period_movie_name)
    period_rm = io.imread(period_movie_path, plugin="tifffile")[:,:,:]    #tzcyx remove first channel

    NFrames, ydim, xdim = period_rm.shape

    # make panda dataframe where rows are space and columns are time

    kymo_rows = int(xdim)
    kymo_columns = int(NFrames/frame_interval_kymo)

    single_periods_forKymo = pd.DataFrame(index = range(kymo_rows), columns = range(kymo_columns))

    for time in range(0,NFrames,frame_interval_kymo): # last number to specify intervals between frames to consider
    
        for space in range(len(single_periods_forKymo.index)):
    
            single_periods_forKymo.iloc[space,time] = period_rm[time,position_kymo,space]

    single_periods_forKymo_float = single_periods_forKymo.astype(float) # change data type of pandas dataframe to float
    
    # plot kymo

    sns.set(font="Helvetica")
    fig,ax = plt.subplots(figsize=(5,8))

    ax = sns.heatmap(single_periods_forKymo_float, cbar = True, cbar_kws={'label': 'period in mins'}, cmap = 'viridis', vmin = 100, vmax = 250)
    ax.set_ylabel('SPACE [pixel]', labelpad = 15, fontsize = 15)
    ax.set_xlabel('TIME [min/10]', labelpad = 15, fontsize = 15)
    # ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 15)
    # ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 15)

    plt.show
    fig.savefig(wdir + '/kymo_pdfs/'+'single_periodKymo_'+strip_string(period_movie_name)+'.png', bbox_inches='tight')
    
    single_periods_forKymo_float_array = np.array(single_periods_forKymo_float)
    single_masked_periodKymo = single_periods_forKymo_float_array * all_mask_intensitiesKymo[index] # masked period kymo
    
    single_masked_periodKymo_df = pd.DataFrame(single_masked_periodKymo)
    
    sns.set(font="Helvetica")
    fig,ax2 = plt.subplots(figsize=(5,8))

    ax2 = sns.heatmap(single_masked_periodKymo_df, cbar = True, cmap = 'viridis', cbar_kws={'label': 'period in mins'}, vmin = 100, vmax = 250)
    ax2.set_ylabel('SPACE [pixel]', labelpad = 15, fontsize = 15)
    ax2.set_xlabel('TIME [min/10]', labelpad = 15, fontsize = 15)
    # ax2.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 15)
    # ax2.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 15)

    plt.show
    fig.savefig(wdir + '/kymo_pdfs/'+'single_periodKymoMasked_'+strip_string(period_movie_name)+'.png', bbox_inches='tight')
    
    all_single_periods_forKymo_float_array.append(single_periods_forKymo_float_array)
    all_single_masked_periodKymo.append(single_masked_periodKymo)

The following script is for generating the phase kymograph and the masked phase kymograph. Outputs are lists containing (a) arrays of phase kymographs of all samples and (b) arrays of their corresponding masked phase kymographs.

In [ ]:
# make phase movie kymo

all_phases_forKymo_float_array = []
all_masked_phaseKymo = []

for index,k in enumerate(phase_movie_names):
    
    phase_movie_name = k.name # the roi movie file name

    # open phase movie as Mrm
    phase_movie_path = os.path.join( wdir, phase_movie_name )
    print('Opening :', phase_movie_name)
    phase_rm = io.imread(phase_movie_path, plugin="tifffile")[:,:,:] #tzcyx remove first channel

    NFrames, ydim, xdim = phase_rm.shape

    # make panda dataframe where rows are space and columns are time

    kymo_rows = int(xdim)
    kymo_columns  = int(NFrames/frame_interval_kymo)

    phases_forKymo = pd.DataFrame(index = range(kymo_rows), columns = range(kymo_columns))

    for time in range(0,NFrames,frame_interval_kymo): # last number to specify intervals between frames to consider
        
        for space in range(len(phases_forKymo.index)):
            
            phases_forKymo.iloc[space,time] = phase_rm[time,position_kymo,space]
    
    phases_forKymo_float = phases_forKymo.astype(float) # change data type of pandas dataframe to float
        
    # plot kymo

    sns.set(font="Helvetica")
    fig,ax2 = plt.subplots(figsize=(5,8))

    ax2 = sns.heatmap(phases_forKymo_float, cbar = True, cmap = 'twilight_shifted', cbar_kws={'label': 'phase in rads'})
    ax2.set_ylabel('SPACE [pixel]', labelpad = 15, fontsize = 15)
    ax2.set_xlabel('TIME [min/10]', labelpad = 15, fontsize = 15)
    # ax2.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 15)
    # ax2.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 15)

    plt.show
    fig.savefig(wdir + '/kymo_pdfs/'+'phaseKymo_'+strip_string(phase_movie_name)+'.png', bbox_inches='tight')
    
    phases_forKymo_float_array = np.array(phases_forKymo_float)
    masked_phaseKymo = phases_forKymo_float_array * all_mask_intensitiesKymo[index] # masked phase kymo
    
    masked_phaseKymo_df = pd.DataFrame(masked_phaseKymo)
    
    sns.set(font="Helvetica")
    fig,ax2 = plt.subplots(figsize=(5,8))

    ax2 = sns.heatmap(masked_phaseKymo_df, cbar = True, cmap = 'twilight_shifted', cbar_kws={'label': 'phase in rads'})
    ax2.set_ylabel('SPACE [pixel]', labelpad = 15, fontsize = 15)
    ax2.set_xlabel('TIME [min/10]', labelpad = 15, fontsize = 15)
    # ax2.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 15)
    # ax2.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 15)

    plt.show
    fig.savefig(wdir + '/kymo_pdfs/'+'phaseKymoMasked_'+strip_string(phase_movie_name)+'.png', bbox_inches='tight')
    
    all_phases_forKymo_float_array.append(phases_forKymo_float_array)
    all_masked_phaseKymo.append(masked_phaseKymo)

In [ ]:
# all_masked_phaseKymo[10][511,119] first sample, element at 512,120 pixel
# plot for several frames per sample
# get vectorial average for multiple samples, with single sample in background
# have summary for vectorial average for multiple samples
# plot slope for half of the spreadout

In [ ]:
half_all_masked_phaseKymo = [] # list of masked phase kymos for half of the spreadout

for sample in range(len(all_masked_phaseKymo)):
    
    half_all_masked_phaseKymo.append(all_masked_phaseKymo[sample][256:,:]) # choose one half of the spreadout, pixel 256 to 512
    
half_all_single_masked_periodKymo = [] # list of masked period kymos for half of the spreadout for one position, index = 256

for sample in range(len(all_single_masked_periodKymo)):
    
    half_all_single_masked_periodKymo.append(all_single_masked_periodKymo[sample][256:,:]) # choose one half of the spreadout, pixel 256 to 512

In [ ]:
# run the analysis; input = all_masked_phaseKymo

all_unwrapped_masked_phase_vector = []
all_normalized_unwrapped_masked_phase_vector = []

all_half_unwrapped_masked_phase_vector = []
all_half_normalized_unwrapped_masked_phase_vector = []

all_phaseSlopes = []
all_phaseFrames = []

for sample in range(len(all_masked_phaseKymo)):
    
    print('Opening : sample ', sample + 1)
    
    # initialize empty lists for slopes and frame numbers
    slopes = []
    frames = []
    
    ydim,xdim = all_masked_phaseKymo[sample].shape # ydim = space, xdim = time

    for frame in range(0,xdim,frame_interval):  # define the frames to measure phase gradient
        
        # extract phase vector from kymo and strip away the zeros (masked values)
        phase_vector = all_masked_phaseKymo[sample][:,frame]
        masked_phase_vector = chop(phase_vector) # chop[0] gives the index and chop[1] gives the value
        half_phase_vector = half_all_masked_phaseKymo[sample][:,frame]
        half_masked_phase_vector = chop(half_phase_vector)
        
        if len(half_masked_phase_vector[0]) > 0: # condition on using the vector... maybe set longer
            
            #unwrap and normalize to the most "posterior" phase value
            unwrapped_masked_phase_vector = np.unwrap(masked_phase_vector[1])
            normalized_unwrapped_masked_phase_vector = unwrapped_masked_phase_vector - unwrapped_masked_phase_vector[0] # [0] is the most "posterior" phase value
            half_unwrapped_masked_phase_vector = np.unwrap(half_masked_phase_vector[1])
            half_normalized_unwrapped_masked_phase_vector = half_unwrapped_masked_phase_vector - half_unwrapped_masked_phase_vector[0]
   
            # append lists for all samples
            all_unwrapped_masked_phase_vector.append(unwrapped_masked_phase_vector)
            all_normalized_unwrapped_masked_phase_vector.append(normalized_unwrapped_masked_phase_vector)
            all_half_unwrapped_masked_phase_vector.append(half_unwrapped_masked_phase_vector)
            all_half_normalized_unwrapped_masked_phase_vector.append(half_normalized_unwrapped_masked_phase_vector)

            # fit linear line on the phase vector to extract slope
            data_x = half_masked_phase_vector[0]
            data_y = half_normalized_unwrapped_masked_phase_vector

            regr = linear_model.LinearRegression()
            regr.fit(data_x.reshape((-1,1)),data_y)
            X = data_x.reshape((-1,1))
            Y = regr.predict(X)

            slope = regr.coef_

            #record the detected slope and the frame number
            frames.append(frame)
            slopes.append(slope)

            ####### for making timelapse of masked phase kymograph
            
            sample_masked_phaseKymo_df = pd.DataFrame(all_masked_phaseKymo[sample])
            
            fig1 = plt.figure(figsize=(15,24))
            ax1 = fig1.add_subplot(111)
            # ax1.set_title("time: "+str(frame*dt)+"min", fontsize = 30, loc = 'right')
            
            ax1 = sns.heatmap(sample_masked_phaseKymo_df, cbar = True, cmap = 'twilight_shifted', cbar_kws={'label': 'phase in rads'})
            ax1.set_ylabel('SPACE [pixel]', labelpad = 15, fontsize = 50)
            ax1.set_xlabel('TIME [min/10]', labelpad = 15, fontsize = 50)
            ax1.axvline(frame, c = 'y', lw = 10)
            ax1.tick_params(axis='both', labelsize=20)
            ax1.figure.axes[-1].yaxis.label.set_size(40)
            cbar = ax1.collections[0].colorbar
            cbar.ax.tick_params(labelsize=40)
            
            fig1.savefig(wdir + '/' + str(frame+2000) + strip_string(phase_movie_names[sample].name) + '_masked_phaseKymo.tif', bbox_inches='tight')
            plt.close(fig1)
            
            #######
            
            ####### for making timelapse of masked phase kymograph for half of the spreadout
            
            half_sample_masked_phaseKymo_df = pd.DataFrame(half_all_masked_phaseKymo[sample])
            
            sns.set(font_scale=2)
            fig2 = plt.figure(figsize=(15,24))
            ax2 = fig2.add_subplot(111)
            # ax1.set_title("time: "+str(frame*dt)+"min", fontsize = 30, loc = 'right')
            
            ax2 = sns.heatmap(half_sample_masked_phaseKymo_df, cbar = True, cmap = 'twilight_shifted', cbar_kws={'label': 'phase in rads'})
            ax2.set_ylabel('SPACE [pixel]', labelpad = 15, fontsize = 50)
            ax2.set_xlabel('TIME [min/10]', labelpad = 15, fontsize = 50)
            ax2.axvline(frame, c = 'y', lw = 10)
            ax2.tick_params(axis='both', labelsize=20)
            ax2.figure.axes[-1].yaxis.label.set_size(40)
            cbar = ax2.collections[0].colorbar
            cbar.ax.tick_params(labelsize=40)
            
            fig2.savefig(wdir + '/' + str(frame+2000) + strip_string(phase_movie_names[sample].name) + '_halfmasked_phaseKymo.tif', bbox_inches='tight')
            plt.close(fig2)
            
            #######
            
            ####### for making timelapse of unwrapped phase gradient
            
            fig3 = plt.figure(figsize=(24,24))
            ax3 = fig3.add_subplot(111)
            ax3.set_title("time: "+str(frame*dt)+"min", fontsize = 50, loc = 'left')
            
            ax3.scatter(masked_phase_vector[0],normalized_unwrapped_masked_phase_vector, c = 'k', s = 500, alpha = 0.3)    
            ax3.set_xlim([0,len(phase_vector)])
            ax3.set_ylim([-9,9])
            ax3.set_ylabel('PHASE [rad]', labelpad = 15, fontsize = 50)
            ax3.set_xlabel('SPACE [pixel]', labelpad = 15, fontsize = 50)
            ax3.tick_params(axis='both', labelsize=50)
            
            fig3.savefig(wdir + '/' + str(frame+2000) + strip_string(phase_movie_names[sample].name) + '_masked_phaseGradient.tif', bbox_inches='tight')
            plt.close(fig3)
            
            #######
            
            ####### for making timelapse of unwrapped phase gradient for half of spreadout
            
            fig4 = plt.figure(figsize=(24,24))
            ax4 = fig4.add_subplot(111)
            ax4.set_title("time: "+str(frame*dt)+"min", fontsize = 50, loc = 'left')
            
            ax4.scatter(half_masked_phase_vector[0],half_normalized_unwrapped_masked_phase_vector, c = 'k', s = 500, alpha = 0.3)    
            ax4.set_xlim([0,len(half_phase_vector)])
            ax4.set_ylim([-9,9])
            ax4.set_ylabel('PHASE [rad]', labelpad = 15, fontsize = 50)
            ax4.set_xlabel('SPACE [pixel]', labelpad = 15, fontsize = 50)
            ax4.tick_params(axis='both', labelsize=50)
            ax4.plot(X,Y, c = 'm', lw = 10, alpha = 0.3) # draw the fit line on plot
            
            fig4.savefig(wdir + '/' + str(frame+2000) + strip_string(phase_movie_names[sample].name) + '_halfmasked_phaseGradient.tif', bbox_inches='tight')
            plt.close(fig4)
            
            #######
            
            ####### for making timelapse of slope of phase gradient for half of spreadout
            
            fig5 = plt.figure(figsize=(24,24))
            ax5 = fig5.add_subplot(111)
            ax5.set_title("SLOPE OF THE PHASE GRADIENT", fontsize = 50, loc = 'center')
            
            ax5.scatter(frame*dt,slope, c = 'k', s = 500, alpha = 1)    
            ax5.set_xlim([0,xdim*dt])
            ax5.set_ylim([-0.15,0.15])
            ax5.set_ylabel('SLOPE OF PHASE GRADIENT [rad/pixel]', labelpad = 15, fontsize = 50)
            ax5.set_xlabel('TIME [min]', labelpad = 15, fontsize = 50)
            ax5.tick_params(axis='both', labelsize = 50)
            
            fig5.savefig(wdir + '/' + str(frame+2000) + strip_string(phase_movie_names[sample].name) + '_slope_phaseGradient.tif', bbox_inches='tight')
            plt.close(fig5)
            
            #######
    
    all_phaseSlopes.append(slopes)
    all_phaseFrames.append(frames)
    
    print('Closing : sample ', sample + 1)
    figsave_to_tif('2*_masked_phaseKymo.tif', str(sample + 1))
    figsave_to_tif('2*_halfmasked_phaseKymo.tif', str(sample + 1))
    figsave_to_tif('2*_masked_phaseGradient.tif', str(sample + 1))
    figsave_to_tif('2*_halfmasked_phaseGradient.tif', str(sample + 1))
    figsave_to_tif('2*_slope_phaseGradient.tif', str(sample + 1))

pd.DataFrame(all_phaseSlopes).to_csv(wdir + '/all_phaseSlopes.csv')
pd.DataFrame(all_phaseFrames).to_csv(wdir + '/all_phaseFrames.csv')

In [ ]:
# run the analysis; input = all_masked_periodKymo

all_single_periodSlopes = []
all_single_freqSlopes = []
all_single_periodFrames = []

for sample in range(len(all_single_masked_periodKymo)):
    
    print('Opening : sample ', sample + 1)
    
    # initialize empty lists for slopes and frame numbers
    slopes = []
    slopes_freq = []
    frames = []
    
    ydim,xdim = all_single_masked_periodKymo[sample].shape # ydim = space, xdim = time

    for frame in range(0,xdim,frame_interval):  # define the frames to measure period gradient
        
        # extract period vector from kymo and strip away the zeros (masked values)
        period_vector = all_single_masked_periodKymo[sample][:,frame]
        masked_period_vector = chop(period_vector) # chop[0] gives the index and chop[1] gives the value
        half_period_vector = half_all_single_masked_periodKymo[sample][:,frame]      
        half_masked_period_vector = chop(half_period_vector)
        
        masked_freq_vector = np.array(masked_period_vector) # copy elements of masked_period_vector into masked_freq_vector
        masked_freq_vector[1] = 2*np.pi/(masked_freq_vector[1]) # replace the periods with frequencies
        half_masked_freq_vector = np.array(half_masked_period_vector)
        half_masked_freq_vector[1] = 2*np.pi/(half_masked_freq_vector[1])
        
        if len(half_masked_period_vector[0]) > 0: # condition on using the vector... maybe set longer

            # fit linear line on the period vector to extract slope
            data_x = half_masked_period_vector[0]
            data_y = half_masked_period_vector[1]
            data_y_freq = half_masked_freq_vector[1]

            regr = linear_model.LinearRegression()
            regr.fit(data_x.reshape((-1,1)),data_y)
            X = data_x.reshape((-1,1)) # data_x.reshape(-1,1) makes data_x a 2D array with same number of rows and 1 column
            Y = regr.predict(X)

            slope = regr.coef_
            
            regr_freq = linear_model.LinearRegression()
            regr_freq.fit(data_x.reshape((-1,1)),data_y_freq)
            X_freq = data_x.reshape((-1,1))
            Y_freq = regr_freq.predict(X_freq)

            slope_freq = regr_freq.coef_

            #record the detected slope and the frame number
            frames.append(frame)
            slopes.append(slope)
            slopes_freq.append(slope_freq)

            ####### for making timelapse of masked period kymograph
            
            sample_masked_periodKymo_df = pd.DataFrame(all_single_masked_periodKymo[sample])
            
            fig1 = plt.figure(figsize=(15,24))
            ax1 = fig1.add_subplot(111)
            # ax1.set_title("time: "+str(frame*dt)+"min", fontsize = 30, loc = 'right')
            
            ax1 = sns.heatmap(sample_masked_periodKymo_df, cbar = True, cmap = 'viridis', cbar_kws={'label': 'period in mins'}, vmin = 100, vmax = 250)
            ax1.set_ylabel('SPACE [pixel]', labelpad = 15, fontsize = 50)
            ax1.set_xlabel('TIME [min/10]', labelpad = 15, fontsize = 50)
            ax1.axvline(frame, c = 'w', lw = 10)
            ax1.tick_params(axis='both', labelsize=20)
            ax1.figure.axes[-1].yaxis.label.set_size(40)
            cbar = ax1.collections[0].colorbar
            cbar.ax.tick_params(labelsize=40)
            
            fig1.savefig(wdir + '/' + str(frame+2000) + strip_string(period_movie_names[sample].name) + '_maskedSingle_periodKymo.tif', bbox_inches='tight')
            plt.close(fig1)
            
            #######
            
            ####### for making timelapse of masked period kymograph for half of the spreadout
            
            half_sample_masked_periodKymo_df = pd.DataFrame(half_all_single_masked_periodKymo[sample])
            
            sns.set(font_scale=2)
            fig2 = plt.figure(figsize=(15,24))
            ax2 = fig2.add_subplot(111)
            # ax1.set_title("time: "+str(frame*dt)+"min", fontsize = 30, loc = 'right')
            
            ax2 = sns.heatmap(half_sample_masked_periodKymo_df, cbar = True, cmap = 'viridis', cbar_kws={'label': 'period in mins'}, vmin = 100, vmax = 250)
            ax2.set_ylabel('SPACE [pixel]', labelpad = 15, fontsize = 50)
            ax2.set_xlabel('TIME [min/10]', labelpad = 15, fontsize = 50)
            ax2.axvline(frame, c = 'w', lw = 10)
            ax2.tick_params(axis='both', labelsize=20)
            ax2.figure.axes[-1].yaxis.label.set_size(40)
            cbar = ax2.collections[0].colorbar
            cbar.ax.tick_params(labelsize=40)
            
            fig2.savefig(wdir + '/' + str(frame+2000) + strip_string(period_movie_names[sample].name) + '_halfmaskedSingle_periodKymo.tif', bbox_inches='tight')
            plt.close(fig2)
            
            #######
            
            ####### for making timelapse of period gradient
            
            fig3 = plt.figure(figsize=(24,24))
            ax3 = fig3.add_subplot(111)
            ax3.set_title("time: "+str(frame*dt)+"min", fontsize = 50, loc = 'left')
            
            ax3.scatter(masked_period_vector[0],masked_period_vector[1], c = 'k', s = 500, alpha = 0.3)    
            ax3.set_xlim([0,len(period_vector)])
            ax3.set_ylim([100,250])
            ax3.set_ylabel('PERIOD [min]', labelpad = 15, fontsize = 50)
            ax3.set_xlabel('SPACE [pixel]', labelpad = 15, fontsize = 50)
            ax3.tick_params(axis='both', labelsize=50)

            fig3.savefig(wdir + '/' + str(frame+2000) + strip_string(period_movie_names[sample].name) + '_maskedSingle_periodGradient.tif', bbox_inches='tight')
            plt.close(fig3)
            
            #######
            
            ####### for making timelapse of period gradient for half of spreadout
            
            fig4 = plt.figure(figsize=(24,24))
            ax4 = fig4.add_subplot(111)
            ax4.set_title("time: "+str(frame*dt)+"min", fontsize = 50, loc = 'left')
            
            ax4.scatter(half_masked_period_vector[0],half_masked_period_vector[1], c = 'k', s = 500, alpha = 0.3)    
            ax4.set_xlim([0,len(half_period_vector)])
            ax4.set_ylim([100,250])
            ax4.set_ylabel('PERIOD [min]', labelpad = 15, fontsize = 50)
            ax4.set_xlabel('SPACE [pixel]', labelpad = 15, fontsize = 50)
            ax4.tick_params(axis='both', labelsize=50)
            ax4.plot(X,Y, c = 'm', lw = 10, alpha = 0.3) # draw the fit line on plot

            fig4.savefig(wdir + '/' + str(frame+2000) + strip_string(period_movie_names[sample].name) + '_halfmaskedSingle_periodGradient.tif', bbox_inches='tight')
            plt.close(fig4)
            
            #######
            
            ####### for making timelapse of period gradient
            
            fig5 = plt.figure(figsize=(24,24))
            ax5 = fig5.add_subplot(111)
            ax5.set_title("time: "+str(frame*dt)+"min", fontsize = 50, loc = 'left')
            
            ax5.scatter(masked_period_vector[0],masked_period_vector[1], c = 'k', s = 500, alpha = 0.3)    
            ax5.set_xlim([0,len(period_vector)])
            ax5.set_ylim([100,250])
            ax5.set_ylabel('PERIOD [min]', labelpad = 15, fontsize = 50)
            ax5.set_xlabel('SPACE [pixel]', labelpad = 15, fontsize = 50)
            ax5.tick_params(axis='both', labelsize=50)
            ax5.axhline(allTs.iloc[frame,sample], c = 'k', lw = 10, ls = ':', alpha = 0.3) # period based on global ROI ~ period of segmentation
            
            fig5.savefig(wdir + '/' + str(frame+2000) + strip_string(period_movie_names[sample].name) + '_maskedSingle_periodGradient_withGlobalPeriod.tif', bbox_inches='tight')
            plt.close(fig5)
            
            #######
            
            ####### for making timelapse of period gradient for half of spreadout
            
            fig6 = plt.figure(figsize=(24,24))
            ax6 = fig6.add_subplot(111)
            ax6.set_title("time: "+str(frame*dt)+"min", fontsize = 50, loc = 'left')
            
            ax6.scatter(half_masked_period_vector[0],half_masked_period_vector[1], c = 'k', s = 500, alpha = 0.3)    
            ax6.set_xlim([0,len(half_period_vector)])
            ax6.set_ylim([100,250])
            ax6.set_ylabel('PERIOD [min]', labelpad = 15, fontsize = 50)
            ax6.set_xlabel('SPACE [pixel]', labelpad = 15, fontsize = 50)
            ax6.tick_params(axis='both', labelsize=50)
            ax6.plot(X,Y, c = 'm', lw = 10, alpha = 0.3) # draw the fit line on plot
            ax6.axhline(allTs.iloc[frame,sample], c = 'k', lw = 10, ls = ':', alpha = 0.3) # period based on global ROI ~ period of segmentation
            
            fig6.savefig(wdir + '/' + str(frame+2000) + strip_string(period_movie_names[sample].name) + '_halfmaskedSingle_periodGradient_withGlobalPeriod.tif', bbox_inches='tight')
            plt.close(fig6)
            
            #######
            
            ####### for making timelapse of slope of period gradient for half of spreadout
            
            fig7 = plt.figure(figsize=(24,24))
            ax7 = fig7.add_subplot(111)
            ax7.set_title("SLOPE OF THE PERIOD GRADIENT", fontsize = 50, loc = 'center')
            
            ax7.scatter(frame*dt,slope, c = 'k', s = 500, alpha = 1)    
            ax7.set_xlim([0,xdim*dt])
            ax7.set_ylim([-0.8,0.8])
            ax7.set_ylabel('SLOPE OF PERIOD GRADIENT [min/pixel]', labelpad = 15, fontsize = 50)
            ax7.set_xlabel('TIME [min]', labelpad = 15, fontsize = 50)
            ax7.tick_params(axis='both', labelsize = 50)
            
            fig7.savefig(wdir + '/' + str(frame+2000) + strip_string(period_movie_names[sample].name) + '_slopeSingle_periodGradient.tif', bbox_inches='tight')
            plt.close(fig7)
            
            #######
            
            ####### for making timelapse of frequency gradient
            
            fig8 = plt.figure(figsize=(24,24))
            ax8 = fig8.add_subplot(111)
            ax8.set_title("time: "+str(frame*dt)+"min", fontsize = 50, loc = 'left')
            
            ax8.scatter(masked_freq_vector[0],masked_freq_vector[1], c = 'k', s = 500, alpha = 0.3)    
            ax8.set_xlim([0,len(period_vector)])
            ax8.set_ylim([(2*np.pi)/250,(2*np.pi)/100])
            ax8.set_ylabel('FREQUENCY [rad/min]', labelpad = 15, fontsize = 50)
            ax8.set_xlabel('SPACE [pixel]', labelpad = 15, fontsize = 50)
            ax8.tick_params(axis='both', labelsize=50)

            fig8.savefig(wdir + '/' + str(frame+2000) + strip_string(period_movie_names[sample].name) + '_maskedSingle_freqGradient.tif', bbox_inches='tight')
            plt.close(fig8)
            
            #######
            
            ####### for making timelapse of frequency gradient for half of spreadout
            
            fig9 = plt.figure(figsize=(24,24))
            ax9 = fig9.add_subplot(111)
            ax9.set_title("time: "+str(frame*dt)+"min", fontsize = 50, loc = 'left')
            
            ax9.scatter(half_masked_freq_vector[0],half_masked_freq_vector[1], c = 'k', s = 500, alpha = 0.3)    
            ax9.set_xlim([0,len(half_period_vector)])
            ax9.set_ylim([(2*np.pi)/250,(2*np.pi)/100])
            ax9.set_ylabel('FREQUENCY [rad/min]', labelpad = 15, fontsize = 50)
            ax9.set_xlabel('SPACE [pixel]', labelpad = 15, fontsize = 50)
            ax9.tick_params(axis='both', labelsize=50)
            ax9.plot(X_freq,Y_freq, c = 'm', lw = 10, alpha = 0.3) # draw the fit line on plot

            fig9.savefig(wdir + '/' + str(frame+2000) + strip_string(period_movie_names[sample].name) + '_halfmaskedSingle_freqGradient.tif', bbox_inches='tight')
            plt.close(fig9)
            
            #######
            
            ####### for making timelapse of frequency gradient
            
            fig10 = plt.figure(figsize=(24,24))
            ax10 = fig10.add_subplot(111)
            ax10.set_title("time: "+str(frame*dt)+"min", fontsize = 50, loc = 'left')
            
            ax10.scatter(masked_freq_vector[0],masked_freq_vector[1], c = 'k', s = 500, alpha = 0.3)    
            ax10.set_xlim([0,len(period_vector)])
            ax10.set_ylim([(2*np.pi)/250,(2*np.pi)/100])
            ax10.set_ylabel('FREQUENCY [rad/min]', labelpad = 15, fontsize = 50)
            ax10.set_xlabel('SPACE [pixel]', labelpad = 15, fontsize = 50)
            ax10.tick_params(axis='both', labelsize=50)
            ax10.axhline(allFreqs.iloc[frame,sample], c = 'k', lw = 10, ls = ':', alpha = 0.3) # frequency based on global ROI ~ frequency of segmentation
            
            fig10.savefig(wdir + '/' + str(frame+2000) + strip_string(period_movie_names[sample].name) + '_maskedSingle_freqGradient_withGlobalFreq.tif', bbox_inches='tight')
            plt.close(fig10)
            
            #######
            
            ####### for making timelapse of frequency gradient for half of spreadout
            
            fig11 = plt.figure(figsize=(24,24))
            ax11 = fig11.add_subplot(111)
            ax11.set_title("time: "+str(frame*dt)+"min", fontsize = 50, loc = 'left')
            
            ax11.scatter(half_masked_freq_vector[0],half_masked_freq_vector[1], c = 'k', s = 500, alpha = 0.3)    
            ax11.set_xlim([0,len(half_period_vector)])
            ax11.set_ylim([(2*np.pi)/250,(2*np.pi)/100])
            ax11.set_ylabel('FREQUENCY [rad/min]', labelpad = 15, fontsize = 50)
            ax11.set_xlabel('SPACE [pixel]', labelpad = 15, fontsize = 50)
            ax11.tick_params(axis='both', labelsize=50)
            ax11.plot(X_freq,Y_freq, c = 'm', lw = 10, alpha = 0.3) # draw the fit line on plot
            ax11.axhline(allFreqs.iloc[frame,sample], c = 'k', lw = 10, ls = ':', alpha = 0.3) # frequency based on global ROI ~ frequency of segmentation
            
            fig11.savefig(wdir + '/' + str(frame+2000) + strip_string(period_movie_names[sample].name) + '_halfmaskedSingle_freqGradient_withGlobalFreq.tif', bbox_inches='tight')
            plt.close(fig11)
            
            #######
            
            ####### for making timelapse of slope of frequency gradient for half of spreadout
            
            fig12 = plt.figure(figsize=(24,24))
            ax12 = fig12.add_subplot(111)
            ax12.set_title("SLOPE OF THE FREQUENCY GRADIENT", fontsize = 50, loc = 'center')
            
            ax12.scatter(frame*dt,slope_freq, c = 'k', s = 500, alpha = 1)    
            ax12.set_xlim([0,xdim*dt])
            ax12.set_ylim([-0.0002,0.0002])
            ax12.set_ylabel('SLOPE OF FREQUENCY GRADIENT [rad/(min$\cdot$pixel)]', labelpad = 15, fontsize = 50)
            ax12.set_xlabel('TIME [min]', labelpad = 15, fontsize = 50)
            ax12.tick_params(axis='both', labelsize = 50)
            
            fig12.savefig(wdir + '/' + str(frame+2000) + strip_string(period_movie_names[sample].name) + '_slopeSingle_freqGradient.tif', bbox_inches='tight')
            plt.close(fig12)
            
            #######
            
    all_single_periodSlopes.append(slopes)
    all_single_freqSlopes.append(slopes_freq)
    all_single_periodFrames.append(frames)
    
    print('Closing : sample ', sample + 1)
    figsave_to_tif('2*_maskedSingle_periodKymo.tif', str(sample + 1))
    figsave_to_tif('2*_halfmaskedSingle_periodKymo.tif', str(sample + 1))
    figsave_to_tif('2*_maskedSingle_periodGradient.tif', str(sample + 1))
    figsave_to_tif('2*_halfmaskedSingle_periodGradient.tif', str(sample + 1))
    figsave_to_tif('2*_maskedSingle_periodGradient_withGlobalPeriod.tif', str(sample + 1))
    figsave_to_tif('2*_halfmaskedSingle_periodGradient_withGlobalPeriod.tif', str(sample + 1))
    figsave_to_tif('2*_slopeSingle_periodGradient.tif', str(sample + 1))
    figsave_to_tif('2*_maskedSingle_freqGradient.tif', str(sample + 1))
    figsave_to_tif('2*_halfmaskedSingle_freqGradient.tif', str(sample + 1))
    figsave_to_tif('2*_maskedSingle_freqGradient_withGlobalFreq.tif', str(sample + 1))
    figsave_to_tif('2*_halfmaskedSingle_freqGradient_withGlobalFreq.tif', str(sample + 1))
    figsave_to_tif('2*_slopeSingle_freqGradient.tif', str(sample + 1))
    
pd.DataFrame(all_single_periodSlopes).to_csv(wdir + '/all_single_periodSlopes.csv')
pd.DataFrame(all_single_freqSlopes).to_csv(wdir + '/all_single_freqSlopes.csv')
pd.DataFrame(all_single_periodFrames).to_csv(wdir + '/all_single_periodFrames.csv')

In [ ]:
# 3D plot of slope of period gradient (position with index = 256), slope of phase gradient, and time

for sample in range(len(all_phaseFrames)):
    
    fig = plt.figure(figsize=(24,24))
    ax = fig.gca(projection='3d')
    
    z = np.array(all_phaseFrames[sample])
    x = np.array(all_single_periodSlopes[sample])
    y = np.array(all_phaseSlopes[sample])

    ax.scatter3D(x, y, z, c=z, cmap='viridis', s = 100, alpha = 1)
    ax.plot3D(x, y, z, 'gray', alpha = 0.5)
    ax.tick_params(axis='both', labelsize = 20)

    ax.set_xlabel('SLOPE OF PERIOD GRADIENT', labelpad = 30, fontsize = 30)
    ax.set_ylabel('SLOPE OF PHASE GRADIENT', labelpad = 30, fontsize = 30)
    ax.set_zlabel('TIME [min/10]', labelpad = 30, fontsize = 30)

    ax.set_xlim([-2.1,2.1])
    ax.set_ylim([-0.15,0.15])
    ax.set_zlim([0,120])

    ax.plot(x, z, 'ko', zdir='y', zs = 0.15, alpha = 0.2)
    ax.plot(y, z, 'ko', zdir='x', zs = -2.1, alpha = 0.2)
    ax.plot(x, y, 'ko', zdir='z', zs = 0, alpha = 0.2)

    plt.show()
    fig.savefig(wdir + '/' + strip_string(period_movie_names[sample].name) + '_singlePeriod3Dplot.pdf', bbox_inches='tight')

In [ ]:
# 3d plot for all samples -- period gradient from slice through one position (index = 256)

fig = plt.figure(figsize=(24,24))
ax = fig.gca(projection='3d')
    
for sample in range(len(all_phaseFrames)):

    z = np.array(all_phaseFrames[sample])
    x = np.array(all_single_periodSlopes[sample])
    y = np.array(all_phaseSlopes[sample])

    ax.scatter3D(x, y, z, c=z, cmap='viridis', s = 100, alpha = 1)
    ax.plot3D(x, y, z, 'gray', alpha = 0.5)
    ax.tick_params(axis='both', labelsize = 20)

    ax.set_xlabel('SLOPE OF PERIOD GRADIENT', labelpad = 30, fontsize = 30)
    ax.set_ylabel('SLOPE OF PHASE GRADIENT', labelpad = 30, fontsize = 30)
    ax.set_zlabel('TIME [min/10]', labelpad = 30, fontsize = 30)

    ax.set_xlim([-2.1,2.1])
    ax.set_ylim([-0.15,0.15])
    ax.set_zlim([0,120])

    ax.plot(x, z, 'ko', zdir='y', zs = 0.15, alpha = 0.05)
    ax.plot(y, z, 'ko', zdir='x', zs = -2.1, alpha = 0.05)
    ax.plot(x, y, 'ko', zdir='z', zs = 0, alpha = 0.05)

plt.show()
fig.savefig(wdir + '/all_singlePeriod3Dplot.pdf', bbox_inches='tight')

In [ ]:
# 3D plot of slope of frequency gradient (position with index = 256), slope of phase gradient, and time

for sample in range(len(all_phaseFrames)):
    
    fig = plt.figure(figsize=(24,24))
    ax = fig.gca(projection='3d')
    
    z = np.array(all_phaseFrames[sample])
    x = np.array(all_single_freqSlopes[sample])
    y = np.array(all_phaseSlopes[sample])

    ax.scatter3D(x, y, z, c=z, cmap='viridis', s = 100, alpha = 1)
    ax.plot3D(x, y, z, 'gray', alpha = 0.5)
    ax.tick_params(axis='both', labelsize = 20)

    ax.set_xlabel('SLOPE OF FREQUENCY GRADIENT', labelpad = 30, fontsize = 30)
    ax.set_ylabel('SLOPE OF PHASE GRADIENT', labelpad = 30, fontsize = 30)
    ax.set_zlabel('TIME [min/10]', labelpad = 30, fontsize = 30)

    ax.set_xlim([-0.0006,0.0006])
    ax.set_ylim([-0.15,0.15])
    ax.set_zlim([0,120])

    ax.plot(x, z, 'ko', zdir='y', zs = 0.15, alpha = 0.2)
    ax.plot(y, z, 'ko', zdir='x', zs = -0.0006, alpha = 0.2)
    ax.plot(x, y, 'ko', zdir='z', zs = 0, alpha = 0.2)

    plt.show()
    fig.savefig(wdir + '/' + strip_string(period_movie_names[sample].name) + '_singleFreq3Dplot.pdf', bbox_inches='tight')

In [ ]:
# 3d plot for all samples

fig = plt.figure(figsize=(24,24))
ax = fig.gca(projection='3d')
    
for sample in range(len(all_phaseFrames)):

    z = np.array(all_phaseFrames[sample])
    x = np.array(all_single_freqSlopes[sample])
    y = np.array(all_phaseSlopes[sample])

    ax.scatter3D(x, y, z, c=z, cmap='viridis', s = 100, alpha = 1)
    ax.plot3D(x, y, z, 'gray', alpha = 0.5)
    ax.tick_params(axis='both', labelsize = 20)

    ax.set_xlabel('SLOPE OF FREQUENCY GRADIENT', labelpad = 30, fontsize = 30)
    ax.set_ylabel('SLOPE OF PHASE GRADIENT', labelpad = 30, fontsize = 30)
    ax.set_zlabel('TIME [min/10]', labelpad = 30, fontsize = 30)

    ax.set_xlim([-0.0006,0.0006])
    ax.set_ylim([-0.15,0.15])
    ax.set_zlim([0,120])

    ax.plot(x, z, 'ko', zdir='y', zs = 0.15, alpha = 0.05)
    ax.plot(y, z, 'ko', zdir='x', zs = -0.0006, alpha = 0.05)
    ax.plot(x, y, 'ko', zdir='z', zs = 0, alpha = 0.05)

plt.show()
fig.savefig(wdir + '/all_singleFreq3Dplot.pdf', bbox_inches='tight')

In [ ]:
colors = cm.viridis(np.linspace(0, 1, len(all_single_periodFrames[0])))

for timepoint in range(len(all_single_periodFrames[0])):
    
    i = timepoint + 1
    
    fig = plt.figure(figsize=(24,24))
    ax = fig.add_subplot(111)
    ax.set_title("SLOPE OF THE FREQUENCY GRADIENT", fontsize = 50, loc = 'center')
    
    for sample in range(len(all_single_periodFrames)):
        
        #j = sample + 1
        
        frames_plot = np.array(all_single_periodFrames[sample][:i])
        slopes_plot = np.array(all_single_freqSlopes[sample][:i])
        
        ax.plot(frames_plot*dt,slopes_plot, c = 'k', alpha = 0.2)
        
        for k in range(len(frames_plot)):
        
            color_plot = colors[k]
            ax.scatter(frames_plot[k]*dt,slopes_plot[k], color = color_plot, s = 500, alpha = 1)
            ax.set_xlim([0,xdim*dt])
            ax.set_ylim([-0.0002,0.0002])
            ax.set_ylabel('SLOPE OF FREQUENCY GRADIENT [rad/(min$\cdot$pixel)]', labelpad = 15, fontsize = 50)
            ax.set_xlabel('TIME [min]', labelpad = 15, fontsize = 50)
            ax.tick_params(axis='both', labelsize = 50)
        
    fig.savefig(wdir + '/' + str(timepoint+2000) + '_all_slopeSingle_freqGradient.tif', bbox_inches='tight')
    

plt.show()
figsave_to_tif('2*_all_slopeSingle_freqGradient.tif', str(sample + 1))

In [ ]:
colors = cm.viridis(np.linspace(0, 1, len(all_single_periodFrames[0])))

for timepoint in range(len(all_single_periodFrames[0])):
    
    i = timepoint + 1
    
    fig = plt.figure(figsize=(24,24))
    ax = fig.add_subplot(111)
    ax.set_title("SLOPE OF THE PERIOD GRADIENT", fontsize = 50, loc = 'center')
    
    for sample in range(len(all_single_periodFrames)):
        
        #j = sample + 1
        
        frames_plot = np.array(all_single_periodFrames[sample][:i])
        slopes_plot = np.array(all_single_periodSlopes[sample][:i])
        
        ax.plot(frames_plot*dt,slopes_plot, c = 'k', alpha = 0.2)
        
        for k in range(len(frames_plot)):
        
            color_plot = colors[k]
            ax.scatter(frames_plot[k]*dt,slopes_plot[k], color = color_plot, s = 500, alpha = 1)
            ax.set_xlim([0,xdim*dt])
            ax.set_ylim([-0.8,0.8])
            ax.set_ylabel('SLOPE OF PERIOD GRADIENT [min/pixel]', labelpad = 15, fontsize = 50)
            ax.set_xlabel('TIME [min]', labelpad = 15, fontsize = 50)
            ax.tick_params(axis='both', labelsize = 50)
        
    fig.savefig(wdir + '/' + str(timepoint+2000) + '_all_slopeSingle_periodGradient.tif', bbox_inches='tight')
    

plt.show()
figsave_to_tif('2*_all_slopeSingle_periodGradient.tif', str(sample + 1))

In [ ]:
colors = cm.viridis(np.linspace(0, 1, len(all_phaseFrames[0])))

for timepoint in range(len(all_phaseFrames[0])):
    
    i = timepoint + 1
    
    fig = plt.figure(figsize=(24,24))
    ax = fig.add_subplot(111)
    ax.set_title("SLOPE OF THE PHASE GRADIENT", fontsize = 50, loc = 'center')
    
    for sample in range(len(all_phaseFrames)):
        
        #j = sample + 1
        
        frames_plot = np.array(all_phaseFrames[sample][:i])
        slopes_plot = np.array(all_phaseSlopes[sample][:i])
        
        ax.plot(frames_plot*dt,slopes_plot, c = 'k', alpha = 0.2)
        
        for k in range(len(frames_plot)):
        
            color_plot = colors[k]
            ax.scatter(frames_plot[k]*dt,slopes_plot[k], color = color_plot, s = 500, alpha = 1)
            ax.set_xlim([0,xdim*dt])
            ax.set_ylim([-0.15,0.15])
            ax.set_ylabel('SLOPE OF PHASE GRADIENT [rad/pixel]', labelpad = 15, fontsize = 50)
            ax.set_xlabel('TIME [min]', labelpad = 15, fontsize = 50)
            ax.tick_params(axis='both', labelsize = 50)
        
    fig.savefig(wdir + '/' + str(timepoint+2000) + '_all_slope_phaseGradient.tif', bbox_inches='tight')
    

plt.show()
figsave_to_tif('2*_all_slope_phaseGradient.tif', str(sample + 1))

In [ ]:
# plot slope frequency versus slope phase

colors = cm.viridis(np.linspace(0, 1, len(all_phaseFrames[0])))

for timepoint in range(len(all_phaseFrames[0])):
    
    i = timepoint + 1
    
    fig1 = plt.figure(figsize=(24,24))
    ax1 = fig1.add_subplot(111)
    ax1.set_title("PHASE GRADIENT VS FREQUENCY GRADIENT", fontsize = 50, loc = 'center')
    
    for sample in range(len(all_phaseFrames)):
        
        #j = sample + 1
        
        frames_plot = np.array(all_phaseFrames[sample][:i])
        freqSlopes_plot = np.array(all_single_freqSlopes[sample][:i])
        phaseSlopes_plot = np.array(all_phaseSlopes[sample][:i])
        
        # ax1.plot(freqSlopes_plot,phaseSlopes_plot, c = 'k', alpha = 0.2)
        
        for k in range(len(frames_plot)):
        
            color_plot = colors[k]
            ax1.scatter(freqSlopes_plot[k],phaseSlopes_plot[k], color = color_plot, s = 500, alpha = 0.5)
            ax1.set_xlim([-0.0002,0.0002])
            ax1.set_ylim([-0.15,0.15])
            ax1.set_ylabel('SLOPE OF PHASE GRADIENT [rad/pixel]', labelpad = 15, fontsize = 50)
            ax1.set_xlabel('SLOPE OF FREQUENCY GRADIENT [rad/(min$\cdot$pixel)]', labelpad = 15, fontsize = 50)
            ax1.tick_params(axis='both', labelsize = 30)
        
    fig1.savefig(wdir + '/' + str(timepoint+2000) + '_all_slope_PhaseSingleFreqGradient.tif', bbox_inches='tight')

plt.show()
figsave_to_tif('2*_all_slope_PhaseSingleFreqGradient.tif', str(sample + 1))    

# plot slope period versus slope phase

colors = cm.viridis(np.linspace(0, 1, len(all_phaseFrames[0])))

for timepoint in range(len(all_phaseFrames[0])):
    
    i = timepoint + 1
    
    fig2 = plt.figure(figsize=(24,24))
    ax2 = fig2.add_subplot(111)
    ax2.set_title("PHASE GRADIENT VS PERIOD GRADIENT", fontsize = 50, loc = 'center')
    
    for sample in range(len(all_phaseFrames)):
        
        #j = sample + 1
        
        frames_plot = np.array(all_phaseFrames[sample][:i])
        periodSlopes_plot = np.array(all_single_periodSlopes[sample][:i])
        phaseSlopes_plot = np.array(all_phaseSlopes[sample][:i])
        
        # ax2.plot(periodSlopes_plot,phaseSlopes_plot, c = 'k', alpha = 0.2)
        
        for k in range(len(frames_plot)):
        
            color_plot = colors[k]
            ax2.scatter(periodSlopes_plot[k],phaseSlopes_plot[k], color = color_plot, s = 500, alpha = 0.5)
            ax2.set_xlim([-0.8,0.8])
            ax2.set_ylim([-0.15,0.15])
            ax2.set_ylabel('SLOPE OF PHASE GRADIENT [rad/pixel]', labelpad = 15, fontsize = 50)
            ax2.set_xlabel('SLOPE OF PERIOD GRADIENT [min/pixel]', labelpad = 15, fontsize = 50)
            ax2.tick_params(axis='both', labelsize = 30)
        
    fig2.savefig(wdir + '/' + str(timepoint+2000) + '_all_slope_PhaseSinglePeriodGradient.tif', bbox_inches='tight')

plt.show()
figsave_to_tif('2*_all_slope_PhaseSinglePeriodGradient.tif', str(sample + 1))    

In [ ]:
# plot slope frequency versus slope phase

colors = cm.viridis(np.linspace(0, 1, len(all_phaseFrames[0])))

for timepoint in range(len(all_phaseFrames[0])):
    
    i = timepoint + 1
    
    fig1 = plt.figure(figsize=(24,24))
    ax1 = fig1.add_subplot(111)
    ax1.set_title("PHASE GRADIENT VS FREQUENCY GRADIENT", fontsize = 50, loc = 'center')
    
    for sample in range(len(all_phaseFrames)):
        
        #j = sample + 1
        sample_label = characters[sample]
        frames_plot = np.array(all_phaseFrames[sample][:i])
        freqSlopes_plot = np.array(all_single_freqSlopes[sample][:i])
        phaseSlopes_plot = np.array(all_phaseSlopes[sample][:i])
        
        # ax1.plot(freqSlopes_plot,phaseSlopes_plot, c = 'k', alpha = 0.2)
        
        for k in range(len(frames_plot)):
        
            if abs(freqSlopes_plot[k] + .0000025) > 0.0002: # not to add labels for values outside the range
                
                time_label = str(k + 1)
                label = sample_label + time_label
                color_plot = colors[k]
                ax1.scatter(freqSlopes_plot[k],phaseSlopes_plot[k], color = color_plot, s = 500, alpha = 0.5)
                ax1.set_xlim([-0.0002,0.0002])
                ax1.set_ylim([-0.15,0.15])
                ax1.set_ylabel('SLOPE OF PHASE GRADIENT [rad/pixel]', labelpad = 15, fontsize = 50)
                ax1.set_xlabel('SLOPE OF FREQUENCY GRADIENT [rad/(min$\cdot$pixel)]', labelpad = 15, fontsize = 50)
                ax1.tick_params(axis='both', labelsize = 30)
                
            else:
                
                time_label = str(k + 1)
                label = sample_label + time_label
                color_plot = colors[k]
                ax1.scatter(freqSlopes_plot[k],phaseSlopes_plot[k], color = color_plot, s = 500, alpha = 0.5)
                ax1.set_xlim([-0.0002,0.0002])
                ax1.set_ylim([-0.15,0.15])
                ax1.set_ylabel('SLOPE OF PHASE GRADIENT [rad/pixel]', labelpad = 15, fontsize = 50)
                ax1.set_xlabel('SLOPE OF FREQUENCY GRADIENT [rad/(min$\cdot$pixel)]', labelpad = 15, fontsize = 50)
                ax1.tick_params(axis='both', labelsize = 30)
                ax1.text(freqSlopes_plot[k] + (.0000025), phaseSlopes_plot[k], label, fontsize=30, alpha = 0.5)
        
    fig1.savefig(wdir + '/' + str(timepoint+2000) + '_all_slope_PhaseSingleFreqGradient_withSampleNumbers.tif', bbox_inches='tight')

plt.show()
figsave_to_tif('2*_all_slope_PhaseSingleFreqGradient_withSampleNumbers.tif', str(sample + 1))    

In [ ]:
# plot slope period versus slope phase

colors = cm.viridis(np.linspace(0, 1, len(all_phaseFrames[0])))

for timepoint in range(len(all_phaseFrames[0])):
    
    i = timepoint + 1
    
    fig2 = plt.figure(figsize=(24,24))
    ax2 = fig2.add_subplot(111)
    ax2.set_title("PHASE GRADIENT VS PERIOD GRADIENT", fontsize = 50, loc = 'center')
    
    for sample in range(len(all_phaseFrames)):
        
        #j = sample + 1
        sample_label = characters[sample]
        frames_plot = np.array(all_phaseFrames[sample][:i])
        periodSlopes_plot = np.array(all_single_periodSlopes[sample][:i])
        phaseSlopes_plot = np.array(all_phaseSlopes[sample][:i])
        
        # ax2.plot(periodSlopes_plot,phaseSlopes_plot, c = 'k', alpha = 0.2)
        
        for k in range(len(frames_plot)):
            
            if abs(periodSlopes_plot[k] + 0.1) > 0.8: # not to add labels for values outside the range
                
                time_label = str(k + 1)
                label = sample_label + time_label
                color_plot = colors[k]
                ax2.scatter(periodSlopes_plot[k],phaseSlopes_plot[k], color = color_plot, s = 500, alpha = 0.5)
                ax2.set_xlim([-0.8,0.8])
                ax2.set_ylim([-0.15,0.15])
                ax2.set_ylabel('SLOPE OF PHASE GRADIENT [rad/pixel]', labelpad = 15, fontsize = 50)
                ax2.set_xlabel('SLOPE OF PERIOD GRADIENT [min/pixel]', labelpad = 15, fontsize = 50)
                ax2.tick_params(axis='both', labelsize = 30)
                
            else:
                
                time_label = str(k + 1)
                label = sample_label + time_label
                color_plot = colors[k]
                ax2.scatter(periodSlopes_plot[k],phaseSlopes_plot[k], color = color_plot, s = 500, alpha = 0.5)
                ax2.set_xlim([-0.8,0.8])
                ax2.set_ylim([-0.15,0.15])
                ax2.set_ylabel('SLOPE OF PHASE GRADIENT [rad/pixel]', labelpad = 15, fontsize = 50)
                ax2.set_xlabel('SLOPE OF PERIOD GRADIENT [min/pixel]', labelpad = 15, fontsize = 50)
                ax2.tick_params(axis='both', labelsize = 30)
                ax2.text(periodSlopes_plot[k] + (.01), phaseSlopes_plot[k], label, fontsize=30, alpha = 0.5)
        
    fig2.savefig(wdir + '/' + str(timepoint+2000) + '_all_slope_PhaseSinglePeriodGradient_withSampleNumbers.tif', bbox_inches='tight')

plt.show()
figsave_to_tif('2*_all_slope_PhaseSinglePeriodGradient_withSampleNumbers.tif', str(sample + 1))

In [ ]:
# plot alpha over time
# plot slope of slope of gradient over time

In [ ]:
colors = cm.viridis(np.linspace(0, 1, len(all_phaseFrames[0])))

for timepoint in range(len(all_phaseFrames[0])):
    
    i = timepoint + 1
    
    fig = plt.figure(figsize=(24,24))
    ax = fig.add_subplot(111)
    ax.set_title("ALPHA OVER TIME", fontsize = 50, loc = 'center')
    
    for sample in range(len(all_phaseFrames)):
        
        #j = sample + 1
        
        frames_plot = np.array(all_phaseFrames[sample][:i])
        phaseSlopes_plot = np.array(all_phaseSlopes[sample][:i])
        freqSlopes_plot = np.array(all_single_freqSlopes[sample][:i])
        alpha = freqSlopes_plot/phaseSlopes_plot
        
        ax.plot(frames_plot*dt,alpha, c = 'k', alpha = 0.2)
        
        for k in range(len(frames_plot)):
        
            color_plot = colors[k]
            ax.scatter(frames_plot[k]*dt,alpha[k], color = color_plot, s = 500, alpha = 1)
            ax.set_xlim([0,xdim*dt])
            ax.set_ylim([-0.02,0.02])
            ax.set_ylabel('ALPHA [1/min]', labelpad = 15, fontsize = 50)
            ax.set_xlabel('TIME [min]', labelpad = 15, fontsize = 50)
            ax.tick_params(axis='both', labelsize = 50)
        
    fig.savefig(wdir + '/' + str(timepoint+2000) + '_all_alpha.tif', bbox_inches='tight')
    
plt.show()
figsave_to_tif('2*_all_alpha.tif', str(sample + 1))

In [ ]:
colors = cm.viridis(np.linspace(0, 1, len(all_phaseFrames[0])))

for timepoint in range(len(all_phaseFrames[0])):
    
    i = timepoint + 1
    
    fig = plt.figure(figsize=(24,24))
    ax = fig.add_subplot(111)
    ax.set_title("ALPHA OVER TIME", fontsize = 50, loc = 'center')
    
    for sample in range(len(all_phaseFrames)):
        
        #j = sample + 1
        
        frames_plot = np.array(all_phaseFrames[sample][:i])
        phaseSlopes_plot = np.array(all_phaseSlopes[sample][:i])
        freqSlopes_plot = np.array(all_single_freqSlopes[sample][:i])
        alpha = freqSlopes_plot/phaseSlopes_plot
        
        ax.plot(frames_plot*dt,alpha, c = 'k', alpha = 0.2)
        
        for k in range(len(frames_plot)):
        
            color_plot = colors[k]
            ax.scatter(frames_plot[k]*dt,alpha[k], color = color_plot, s = 500, alpha = 1)
            ax.set_xlim([0,xdim*dt])
            ax.set_ylim([-0.15,0.15])
            ax.set_ylabel('ALPHA [1/min]', labelpad = 15, fontsize = 50)
            ax.set_xlabel('TIME [min]', labelpad = 15, fontsize = 50)
            ax.tick_params(axis='both', labelsize = 50)
        
    fig.savefig(wdir + '/' + str(timepoint+2000) + '_all_alpha_zoomOut.tif', bbox_inches='tight')
    
plt.show()
figsave_to_tif('2*_all_alpha_zoomOut.tif', str(sample + 1))